In [7]:
# !pip install fosforml

In [5]:
#Snowpark lib
from snowflake.snowpark import Session

# Data Science Libs
import numpy as np
import pandas as pd

# create_temp_table warning suppresion
import warnings; warnings.simplefilter('ignore')

#ConfigParser to read ini file
import configparser

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cluster import DBSCAN
import pickle

from fosforml import *
from fosforml import scoring_func
from fosforml.constants import MLModelFlavours
import requests

np.random.seed(0)

config = configparser.ConfigParser()
config.read("/notebooks/notebooks/credentials.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
with open('churn_model.pkl','rb') as f:  ## use rb while reading the fie
    churn_model = pickle.load(f)

In [21]:
@scoring_func
def score(model, request):
    payload_dict = eval(request.json["payload"])
    data = pd.DataFrame.from_dict(payload_dict)
    frame = data[['MEMBER_GENDER', 'MEMBER_STATE', 'MEMBER_CONTACT_VERIFIED','FUND_TOTAL_ASSETS','FUND_RETURN_TARGET_PERCENTAGE',
            'INVESTMENT_RISK_CATEGORY', 'CASH_BENCHMARK_ALLOCATION','FIXED_INCOME_BENCHMARK_ALLOCATION',
            'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION','INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION',
            'UNLISTED_EQUITY_BENCHMARK_ALLOCATION', 'EQUITY_BENCHMARK_ALLOCATION','PROPERTY_BENCHMARK_ALLOCATION',
            'INFRA_BENCHMARK_ALLOCATION','COMMODITIES_BENCHMARK_ALLOCATION', 'OTHERS_BENCHMARK_ALLOCATION','FUND_RISK_LEVEL', 
            'FUND_RISK_CATEGORY','NEG_NETRETURN_SINCE_INCEPTION', 'YEAR_1_RETURNS', 'YEAR_3_RETURNS','YEAR_5_RETURNS',
            'YEAR_7_RETURNS','YEAR_10_RETURNS', 'SUPER_FEES','PENSION_FEES',
            'INVESTMENT_AGE_GROUP', 'RETIREMENT_AGE_GROUP', 'TOTAL_FUNDS_INVESTED','CHURN_FLAG']].copy()
#     result = model.predict(frame)
#     prediction = pd.DataFrame({"CHURN_PREDICTION":result})
#     prediction["CHURN_PREDICTION"] = prediction["CHURN_PREDICTION"].apply(lambda x: "CHURN" if x == 1 else "NOT CHURN")
#     probability = model.predict_proba(frame)
#     prediction["Probability"] = [round(k[1],4) for k in probability]
#     prediction = str(prediction.to_dict())
#     return prediction

In [7]:
df = session.table("MEMBER_FUNDS_ENRICHED_DETAILS").to_pandas()

In [ ]:
df = df.drop(["MEMBER_DOB_DT", 'ALLOCATION_DT', "CHURN_DT", "RETIREMENT_DT"])

In [16]:
payload = df.head(1).to_dict()
req = requests.Request()
req.json = {"payload":str(payload)}
y = req


In [27]:
y.json["payload"]

"{'MEMBER_ID': {0: 'MID108406'}, 'MEMBER_NAME': {0: 'Maile Treston'}, 'MEMBER_EMPLOYMENT': {0: 'Charles Darwin University'}, 'MEMBER_GENDER': {0: 'Male'}, 'MEMBER_CITY_TOWN': {0: 'Lakes Entrance'}, 'MEMBER_STATE': {0: 'Victoria'}, 'MEMBER_CONTACT_VERIFIED': {0: 'Y'}, 'FUND_ID': {0: 'FID000044'}, 'CHURN_REASON': {0: 'NOT APPLICABLE'}, 'LATITUDE': {0: -37.8667}, 'LONGITUDE': {0: 147.9833}, 'COUNTRY': {0: 'Australia'}, 'COUNTRYCODE': {0: 'AU'}, 'CAPITAL': {0: 'NOT APPLICABLE'}, 'POPULATION': {0: 6071}, 'FUND_TOTAL_ASSETS': {0: 5866.0}, 'FUND_RETURN_TARGET_PERCENTAGE': {0: 3.247619048}, 'INVESTMENT_RISK_LEVEL': {0: 3.59047619}, 'INVESTMENT_RISK_CATEGORY': {0: 'Medium-Low'}, 'CASH_BENCHMARK_ALLOCATION': {0: 4.047619048}, 'FIXED_INCOME_BENCHMARK_ALLOCATION': {0: 9.761904762}, 'DOMESTIC_LISTED_EQUITY_BENCHMARK_ALLOCATION': {0: 24.52380952}, 'INTERNATIONAL_LISTED_EQUITY_BENCHMARK_ALLOCATION': {0: 25.19047619}, 'UNLISTED_EQUITY_BENCHMARK_ALLOCATION': {0: 0.0}, 'EQUITY_BENCHMARK_ALLOCATION': {0:

In [22]:
yo = score(churn_model, y)
yo

NameError: name 'datetime' is not defined